In [ ]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
import cmdstanpy
cmdstanpy.install_cmdstan()


In [ ]:
# ======================
# 1. Simulated Dataset
# ======================
np.random.seed(42)
## Create date ranges: two years of daily data
dates = pd.date_range(start='2022-01-01', end='2022-12-31', freq='D')
## Temperature simulation
days = np.arange(len(dates))
trend = 0.01 * days
seasonality = 10 * np.sin(2 * np.pi * days / 365)  # yearly cycle
noise = np.random.normal(0, 2, len(dates))         # random variations
temperature = 20 + trend + seasonality + noise
## Create Dataset
df = pd.DataFrame({
    'Date': dates,
    'Temperature': temperature
})

## Visualization
plt.figure(figsize=(12,5))
plt.plot(df["Date"], df["Temperature"], label="Simulated Temperature")
plt.title("Daily Temperature (simulated)")
plt.xlabel("Date")
plt.ylabel("°C")
plt.legend()
plt.show()

In [ ]:
# ======================
# 2. ARIMA Model
# ======================
serie = df['Temperature']
# ARIMA settings
arima_model = ARIMA(serie, order=(2,1,1)).fit()  # (p,d,q)
arima_prediction = arima_model.get_prediction(
    start=len(serie)-90, end=len(serie)+30
).predicted_mean

# Visualize ARIMA prediction
plt.figure(figsize=(12,5))
plt.plot(df["Date"], serie, label="Real Temperature")
plt.plot(df["Date"].iloc[-90:], arima_prediction[:-31],
         label="ARIMA (last 3 months)", color="orange")
plt.plot(pd.date_range(start=df["Date"].iloc[-1], periods=31, freq="D"),
         arima_prediction[-31:], label="ARIMA (prediction)",
         linestyle="--", color="red")
plt.title("ARIMA Temperature Forecast")
plt.xlabel("Date")
plt.ylabel("Temperature")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# ======================
# 3. Prophet Model
# ======================
## Preparing data
df_prophet = df.rename(columns={'Date':'ds','Temperature':'y'})
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])
df_prophet['y'] = df_prophet['y'].astype(float)

## Train/test split (simulate forecasting 3 months ahead)
train = df_prophet.iloc[:-90]
test = df_prophet.iloc[-90:]

prophet_model = Prophet(interval_width=0.95)
prophet_model.add_seasonality(name='yearly_temp', period=365.25, fourier_order=10)
prophet_model.fit(train)

## Forecast next 90 days
future = prophet_model.make_future_dataframe(periods=90, freq="D")
forecast = prophet_model.predict(future)

# Evaluate performance
y_true = test['y'].values
y_pred = forecast.set_index('ds').loc[test['ds'], 'yhat'].values
print("=== Prophet Evaluation (last 90 days) ===")
print("MAE:", mean_absolute_error(y_true, y_pred))
print("RMSE:", mean_squared_error(y_true, y_pred, squared=False))

# Plot Prophet forecast
prophet_model.plot(forecast)
plt.title('Prediction by Prophet')
plt.show()

# Prophet components
prophet_model.plot_components(forecast)
plt.show()

# Interactive Plotly forecast
plot_plotly(prophet_model, forecast).show()
plot_components_plotly(prophet_model, forecast).show()


In [ ]:
# ======================
# 4. Comparison: ARIMA vs Prophet
# ======================
plt.figure(figsize=(12,5))
plt.plot(df["Date"], df["Temperature"], label="Real Temperature")
plt.plot(pd.date_range(start=df["Date"].iloc[-90], periods=90, freq="D"),
         arima_prediction[:-31], label="ARIMA (last 3 months)", color="orange")
plt.plot(pd.date_range(start=df["Date"].iloc[-1], periods=31, freq="D"),
         arima_prediction[-31:], label="ARIMA (prediction)", linestyle="--", color="red")
plt.plot(forecast['ds'], forecast['yhat'], label="Prophet prediction", color="green", alpha=0.7)
plt.title("Temperature Forecast Comparison: ARIMA vs Prophet")
plt.xlabel("Date")
plt.ylabel("Temperature")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()